These are common functions and libraries that we are going to need to use throughout the labs.
There are two key files:
1. .env - a file containing configuration information that is loaded into environment variables.
2. common.py - common python functions that we will use to learn about the API.

In [1]:
# common libraries
from dotenv import load_dotenv
import os
from os import environ
import openai
from icecream import ic

# load our environment file
load_dotenv()

# define our API Key
openai.api_key = os.getenv("openai_api_key")

# Exercise 1 

Support Functions

We keep a list of all the designations of all the openAI models that are valid.

In [102]:
from typing import List, Dict, Any

open_ai_models = ['text-search-babbage-doc-001', 'gpt-3.5-turbo-16k-0613', 'gpt-3.5-turbo-0613', 'curie-search-query', \
                'gpt-3.5-turbo', 'gpt-3.5-turbo-16k', 'text-search-babbage-query-001', 'babbage', 'babbage-search-query', \
                'text-babbage-001', 'fanw-json-eval', 'whisper-1', 'text-similarity-davinci-001', 'gpt-4', 'davinci',\
                'davinci-similarity', 'code-davinci-edit-001', 'curie-similarity', 'babbage-search-document', 'curie-instruct-beta',\
                'text-search-ada-doc-001', 'davinci-instruct-beta', 'text-similarity-babbage-001', 'text-search-davinci-doc-001', \
                'gpt-4-0314', 'babbage-similarity', 'davinci-search-query', 'text-similarity-curie-001', 'text-davinci-001', \
                'text-search-davinci-query-001', 'ada-search-document', 'ada-code-search-code', 'babbage-002', 'gpt-4-0613', \
                'davinci-002', 'davinci-search-document', 'curie-search-document', 'babbage-code-search-code', \
                'text-search-ada-query-001', 'code-search-ada-text-001', 'babbage-code-search-text', 'code-search-babbage-code-001', \
                'ada-search-query', 'ada-code-search-text', 'text-search-curie-query-001', 'text-davinci-002', 'text-embedding-ada-002', \
                'text-davinci-edit-001', 'code-search-babbage-text-001', 'gpt-3.5-turbo-instruct-0914', 'ada', 'text-ada-001', \
                'ada-similarity', 'code-search-ada-code-001', 'text-similarity-ada-001', 'gpt-3.5-turbo-0301', \
                'gpt-3.5-turbo-instruct', 'text-search-curie-doc-001', 'text-davinci-003', 'text-curie-001', 'curie']


A valid message for OpenAI has two features
1. It is valid JSON.
1. It has a "role".
1. It has a "message".

We can check this with `is_valid_message`

In [98]:
def is_valid_message(message: Dict[str, Any]) -> bool:
    """
    Check if a single message dictionary has the correct format to be sent to OpenAI.

    Args:
        message (Dict[str, Any]): A message dictionary with 'role' (str) and 'content' (str) keys.

    Returns:
        bool: True if the message is in the correct format, False otherwise.
    """
    # Check if the message dictionary has 'role' and 'content' keys of the correct types.
    if isinstance(message, dict) and 'role' in message and 'content' in message:
        if isinstance(message['role'], str) and isinstance(message['content'], str):
            return True
    return False


It is very common to have to check multiple messages, not just one.

We can check them all with `are_valid_messages`

In [99]:
def are_valid_messages(messages: List[Dict[str, Any]]) -> bool:
    """
    Check if a list of messages is in the correct format to be sent to OpenAI.

    Args:
        messages (List[Dict[str, Any]]): A list of message dictionaries.

    Returns:
        bool: True if all messages are in the correct format, False otherwise.
    """
    return all(is_valid_message(message) for message in messages)


Most chat interactions have a few parts:
1.  A list of messages to be processed together.
1. An ID for the openAI model to be used.
1. How creattive you want the generation to be.
1. The maximum number of tokens you want to use.

We encapsulate this in `simple_chat`, and get back a JSON object with a number of different values.

In [100]:
def simple_chat(messages: List[Dict[str, Any]], model: str = 'gpt-3.5-turbo', temperature: float = 0.9, max_tokens: int = 1024) -> str:
    """
    Conduct a simple chat conversation using OpenAI's GPT-3 model.

    Args:
        messages (List[Dict[str, Any]]): A list of message dictionaries, where each dictionary contains a 'role' (str)
            and 'content' (str) key-value pair representing the role of the message sender (e.g., 'system', 'user', 'assistant')
            and the content of the message.
        model (str, optional): The OpenAI model to use (default is 'gpt-3.5-turbo').
        temperature (float, optional): Controls the randomness of the response. Higher values (e.g., 0.9) make the output more random,
            while lower values (e.g., 0.2) make it more deterministic. Default is 0.9.
        max_tokens (int, optional): The maximum length of the response, measured in tokens. Default is 1024 tokens.

    Returns:
        str: The response generated by the GPT-3 model.

    Raises:
        ValueError: If the input messages are not in the correct format.

    Example:
        messages = [
            {'role': 'system', 'content': 'You are a helpful assistant.'},
            {'role': 'user', 'content': 'What's the weather like today?'},
        ]
        response = simple_chat(messages)
        print(response)  # Print the generated response.
    """

    if not messages:
        raise ValueError("Input messages list cannot be empty.")

    # Check if all messages are in the correct format.
    if not are_valid_messages(messages):
        raise ValueError("Input messages must be in the format [{'role': str, 'content': str}, ...]")

    if model not in open_ai_models:
        raise ValueError(f"{model} is not a valid model name.")

    # Send the messages to OpenAI and get the response
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )

    return response


It can be very useful to look at the actual details of the response.

`show_response_detail` is a convienience function that outputs the details to the screen.

In [101]:
def show_response_detail(response: openai.openai_object.OpenAIObject):
    """
    Extracts and displays details of the first message choice from an OpenAI response object.

    This function is designed to work with response objects returned by OpenAI's language models,
    specifically with choices that contain messages with 'role' and 'content' attributes.

    Args:
        response (openai.openai_object.OpenAIObject): The OpenAI response object containing message choices.

    Returns:
        None

    Example:
        response = openai.Completion.create(
            model="gpt-3.5-turbo",
            prompt="Translate the following English text to French: 'Hello, world.'"
        )
        response_detail(response)
    """
    
    ic({response.choices[0].message.role})
    ic({response.choices[0].message.content})
    ic({response.usage.prompt_tokens})
    ic({response.usage.completion_tokens})
    ic({response.usage.total_tokens})

# Exercise 2

Summarizing a Message

A good job for Generative AI is generating text.  Here is a code example of exactly how you can do this with OpenAI.

** Notes **

1. We are using two different roles to definte the messages.
1. The response object contains a lot of interesting information in it.

In [103]:
# these are arguments that can be pre-defined and passed to the simple_chat function.
# they can be changed as needed. 
simple_chat_args = {
    'temperature': 0,
    'model': 'gpt-3.5-turbo',
    'max_tokens': 2000,
}

This example will:
1. Declare a long message.
1. Declare a system message.
1. Pass the long message as a user.
1. Get back a response as a summary.

In [92]:
# a really long message to deal with
long_message = """Jupiter is the fifth planet from the Sun and the largest in the Solar System. 
It is a gas giant with a mass one-thousandth that of the Sun, but two-and-a-half times that of all the other planets in the Solar System
 combined. Jupiter is one of the brightest objects visible to the naked eye in the night sky, and has been known to ancient civilizations 
 since before recorded history. It is named after the Roman god Jupiter. When viewed from Earth, Jupiter can be bright enough for its 
 reflected light to cast visible shadows, and is on average the third-brightest natural object in the night sky after the Moon and Venus."""

# build our messages to send to openAI.  These should be well formed JSON with a ROLE and CONTENT
system_message = {"role":"system", "content":"Summarize content you are provided."}
user_message = {"role":"user", "content":long_message}
# send the information to OpenAI and get back a response
summary_response = simple_chat(messages=[system_message, user_message], **simple_chat_args)

In [ ]:
# You can view the contents of any variable by putting it at the end of any cell.
summary_response

In [ ]:
# show_response_detail will give us the same information but formatted a bit more nicely.
show_response_detail(summary_response)

## Exercise 1 - 

Change the summary so that the text is not more than two sentences and is appropriate for a second grader.

Don't forget to change the `system_message` variable.

In [ ]:
# Change the system message.
system_message = {"role":"system", "content":"Summarize content you are provided in 2 sentences or less."}
user_message = {"role":"user", "content":long_message}
# send the information to OpenAI and get back a response
summary_response = simple_chat(messages=[system_message, user_message], **simple_chat_args)
show_response_detail(summary_response)

# Exercise 3

Classification

Another good job for Generative AI is generating classification, or putting things into buckets.  

In [121]:
simple_chat_args = {
    'temperature': 0,
    'model': 'gpt-3.5-turbo',
    'max_tokens': 2000,
}

# Define a list containing names of flowers
flowers = ["Sunflower", "Carnation", "Bluebonnet"]

# Define a list containing names of people
people = ["Alice", "Bob", "Carla"]

# Combine all the individual lists (flowers, people) into one comprehensive list
everything = flowers + people

# Set up an instruction for the system to classify the items in the 'everything' list
instructions = "Classify as one or more types: flower, people, or other."
system_message = {"role": "system", "content": instructions}

# Iterate over each item in the 'everything' list
for item in everything:
    
    # Construct a user message for each item, prompting its classification
    user_message = {"role": "user", "content": f"Classify this: {item}"}
    
    # Send the system and user messages to and get back a classification response
    classification_response = simple_chat(messages=[system_message, user_message], **simple_chat_args)
    
    # Extract the content of the response which contains the classification
    classification = classification_response.choices[0].message.content
    
    # this should look like: ic| item: <Item Name>, classification: <Item Classification>
    # Print (or log) the item and its classification
    ic(item, classification)


ic| item: 'Sunflower', classification: 'flower'
ic| item: 'Carnation', classification: 'flower'
ic| item: 'Bluebonnet', classification: 'flower'
ic| item: 'Alice', classification: 'people'
ic| item: 'Bob', classification: 'people'
ic| item: 'Carla', classification: 'people'


# Exercise 4

Change the classifications to include different types of food and add at least 3 food items to be classified.

In [ ]:
simple_chat_args = {
    'temperature': 0,
    'model': 'gpt-3.5-turbo',
    'max_tokens': 2000,
}

# Define a list containing names of flowers
flowers = ["Sunflower", "Carnation", "Bluebonnet"]

# Define a list containing names of people
people = ["Alice", "Bob", "Carla"]

# 
food = []

# Combine all the individual lists (flowers, people, food - don't forget to add FOOD to the list) into one comprehensive list
everything = flowers + people 

# Set up an instruction for the system to classify the items in the 'everything' list
# don't forget to change the instructions to include FOOD
instructions = "Classify as one or more types: flower, people, or other."
system_message = {"role": "system", "content": instructions}

# Iterate over each item in the 'everything' list
for item in everything:
    
    # Construct a user message for each item, prompting its classification
    user_message = {"role": "user", "content": f"Classify this: {item}"}
    
    # Send the system and user messages to and get back a classification response
    classification_response = simple_chat(messages=[system_message, user_message], **simple_chat_args)
    
    # Extract the content of the response which contains the classification
    classification = classification_response.choices[0].message.content
    
    # this should look like: ic| item: <Item Name>, classification: <Item Classification>
    # Print (or log) the item and its classification
    ic(item, classification)

# Lab 0 - Example 3

## Generating Data

Another good job for Generative AI is generating realistic looking data, either structured or unstructured. 

In [ ]:
# define the arguments we are going to use
simple_chat_args = {
    'temperature': 0.5,
    'model': 'gpt-3.5-turbo',
    'max_tokens': 2000,
}

# define some parameters for this story
subject = "bears"
hero = "Lilly the frog"
location = "The Moon"

# build the story description from the parameters
description = f"Generate a three paragraph story about {subject} that takes place in {location} with a hero named {hero}."

# create the messages we are going to use to create the story.
system_message = {"role":"system", "content":"You are a helpful assistant who tells creative stories for children."}
user_message = {"role":"user", "content": description}

# send the information to OpenAI and get back a response
story_response = simple_chat(messages=[system_message, user_message], **simple_chat_args)
# extract the response from the larger JSON object that was returned
story = story_response.choices[0].message.content

ic(story)

## Exercise 3

Create your own story, replace with your own subject, hero, description, and any other variables you wish to add.

In [ ]:
# define the arguments we are going to use
simple_chat_args = {
    'temperature': 0.5,
    'model': 'gpt-3.5-turbo',
    'max_tokens': 2000,
}

# Don't forget to change these.
subject = ""
hero = ""
location = ""

# build the story description from the parameters
description = f"Generate a three paragraph story about {subject} that takes place in {location} with a hero named {hero}."

# create the messages we are going to use to create the story.
system_message = {"role":"system", "content":"You are a helpful assistant who tells creative stories for children."}
user_message = {"role":"user", "content": description}

# send the information to OpenAI and get back a response
story_response = simple_chat(messages=[system_message, user_message], **simple_chat_args)
# extract the response from the larger JSON object that was returned
story = story_response.choices[0].message.content

ic(story)

## Exercise 4

Create a CSV with random data representing employees.

In [ ]:
# define the arguments we are goint to use
simple_chat_args = {
    'temperature': 0.8,
    'model': 'gpt-3.5-turbo',
    'max_tokens': 2000,
}

description = "Generate a list of 10 people who work in an office. Include id, name, email address, and salary."

# create the messages we are going to use to create the story.
system_message = {"role":"system", "content": "You are a helpful assistant who generates CSV data for spreadsheets"}
user_message = {"role":"user", "content": description}

# send the information to OpenAI and get back a response
csv_response = simple_chat(messages=[system_message, user_message])
# extract the response from the larger JSON object that was returned
csv = csv_response.choices[0].message.content

ic(csv)

Add a column to the CSV for each employees hire date and title.
Modify the settings so that you will get the same results every time you run the cell

In [ ]:
# define the arguments we are going to use
simple_chat_args = {
    'temperature': 0.5,
    'model': 'gpt-3.5-turbo',
    'max_tokens': 2000,
}

# don't forget to change the description with the extra fields.
description = "Generate a list of 10 people who work in an office. Include id, name, email address, salary, hire date, and title."

# create the messages we are going to use to create the story.
system_message = {"role":"system", "content":"You are a helpful assistant who generates CSV data for spreadsheets"}
user_message = {"role":"user", "content": description}

# send the information to OpenAI and get back a response
csv_response = simple_chat(messages=[system_message, user_message], **simple_chat_args)
# extract the response from the larger JSON object that was returned
csv = csv_response.choices[0].message.content

ic(csv)